## Part 1: From Twitter API

In [1]:
# import Dependencies
import pandas as pd
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt
import requests
import json
import re
import datetime as dt
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()

In [2]:
# Calculate the date of week ago, since our standard user package only allows 7 days of tweets access
last_week=(dt.datetime.now() - dt.timedelta(days=7)).strftime("%Y-%m-%d")

In [3]:
# Ask for a date in the past 7 days and the app below will base on the date entered"
end_date=input("Which date of the last 7 days would you like to end?  YYYY-MM-DD  ")

Which date of the last 7 days would you like to end?  YYYY-MM-DD  2019-04-16


In [4]:
# Display time frame
print(f'With Standard Account, you have chosen the time frame from {last_week} to {end_date}.')
print("Update to Premium Account to see more.")

With Standard Account, you have chosen the time frame from 2019-04-10 to 2019-04-16.
Update to Premium Account to see more.


In [5]:
# set up args and load API keys into it to generate bearer_token
premium_search_args = load_credentials("twitter_keys.yaml",
                                       yaml_key="search_tweets_premium",
                                       env_overwrite=False)

In [6]:
# set up token and headers for requests.get
premium_search_args.get('bearer_token')
headers={'Authorization': 'Bearer ' + premium_search_args.get('bearer_token')}

In [7]:
# set up a list of keywords, which can be enriched in the future
keyword_list=["nextdaytrade","stocktwits","stockwatch","stockmarket",
              "stocktrend",'biostocks','biostock','stockstream','daystock',
              'stocklist','stockwizard','toptraders','toptrader','topstock',
             'wallstreet','newyorkstock','nyse','bloomberg','newyorktimes',
             'protrader','protraders','stockpick','stockfinance','stockpicks']

In [8]:
# multiple for loops to extract tickers from tweets allowed to access and use Regex to search only tickers containing $ sign
extracted_tickers=[]
for keyword in keyword_list:
    url='https://api.twitter.com/1.1/search/tweets.json?q='+keyword+'&until='+end_date+"lang=en"
    rjson = requests.get(url,headers=headers).json()
    
    for i in range(len(rjson['statuses'])):
        extracted_text = rjson['statuses'][i]['text']
        extracted_list=re.findall(r'\$\b[A-Z]{2,4}\b[.!?]?',extracted_text)
        looplist=[]
        for t in extracted_list:
            looplist.append(t[1:])
        extracted_tickers=extracted_tickers+looplist

In [9]:
# clean the tickers for period sign
new_tickers=[]
for t in extracted_tickers:
    new_t=t.replace('.','')
    new_tickers.append(new_t)
new_tickers

['PCG',
 'DIS',
 'CGC',
 'BA',
 'GS',
 'NFLX',
 'UNH',
 'ROKU',
 'AAPL',
 'TSLA',
 'LYFT',
 'BA',
 'UNH',
 'TSLA',
 'BBBY',
 'CGC',
 'AAPL',
 'LYFT',
 'LYFT',
 'BBBY',
 'AAPL',
 'TSLA',
 'ROKU',
 'NVDA',
 'MNST',
 'BA',
 'BIDU',
 'AAPL',
 'TSLA',
 'ROKU',
 'NBEV',
 'AAPL',
 'BA',
 'ROKU',
 'TSLA',
 'CF',
 'XLP',
 'BB',
 'TSLA',
 'VZ',
 'TER',
 'PREV',
 'CCBG',
 'WWE',
 'ADNT',
 'BOOM',
 'PCG',
 'CYBR',
 'CVNA',
 'YETI',
 'PUMP',
 'RDN',
 'NOK',
 'VALE',
 'NIO',
 'ABEV',
 'XHR',
 'FCSC',
 'XON',
 'BHVN',
 'ADVM',
 'ACST',
 'ADRO',
 'INSY',
 'ADVM',
 'ADVM',
 'NOK',
 'VALE',
 'NIO',
 'ABEV',
 'XHR',
 'NYSE',
 'CCBG',
 'WWE',
 'ADNT',
 'BOOM',
 'PCG',
 'CYBR',
 'CVNA',
 'YETI',
 'PUMP',
 'RDN',
 'NOK',
 'VALE',
 'NIO',
 'ABEV',
 'XHR',
 'PEP',
 'RLGT',
 'DIS',
 'SPY',
 'WUHN',
 'BOP',
 'ESNC',
 'WUHN',
 'BOP',
 'ESNC',
 'WUHN',
 'BOP',
 'ESNC',
 'WUHN',
 'BOP',
 'ESNC',
 'WUHN',
 'BOP',
 'ESNC',
 'WUHN',
 'BOP',
 'ESNC',
 'WUHN',
 'BOP',
 'ESNC',
 'WUHN',
 'BOP',
 'ESNC',
 'GIB',
 'GIB',


In [10]:
# convert the list into dataframe to help sort
df=pd.DataFrame({"ticker":new_tickers})

# count ticker frequency and sort with descending order, take the top 10 tickers
df1 = df.ticker.value_counts()
top_ten_df=df1[0:10].reset_index()
top_ten_ticker=list(top_ten_df['index'])

# a display of our list of tickers
top_ten_ticker

['ESNC', 'WUHN', 'BOP', 'TSLA', 'AAPL', 'ROKU', 'BA', 'NOK', 'LYFT', 'PCG']

## Part 2: Yahoo Finance Scrape (using results from Part1)

In [12]:
# set up path
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [13]:
# Build query Url and List of Stocks ticker
base_url = "https://finance.yahoo.com/quote/"
tickers = top_ten_ticker

In [14]:
# create and initialize list 
close_price_list = []
volume_list = []
mkt_cap_list = []
earnings_Date_list = []
eps_list = []

In [24]:
for ticker in tickers:
    try:
        # Visit Yahoo finance summary page
        url1 = base_url + ticker + '?p=' + ticker + '&.tsrc=fin-srch'
    #     print(url1)
        browser.visit(url1)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")
        summary = soup.find('table', class_="W(100%)").find_all(class_="Trsdu(0.3s) ")
        market_info = soup.find('table', class_="W(100%) M(0) Bdcl(c)").find_all(class_="Trsdu(0.3s) ")
        #print(summary)
        #type(summary)
        close_price = soup.find('div', class_="D(ib) Mend(20px)").text
    #     print(close_price.split('+')[0])
        close_price = summary[0].contents[0]
        close_price_list.append(close_price)
        volume = summary[4].contents[0]
        volume_list.append(volume)
        market_cap = market_info[0].contents[0]
        mkt_cap_list.append(market_cap)
        earnings_Date = market_info[4].contents[0]
        earnings_Date_list.append(earnings_Date)
        eps = market_info[3].contents[0]
        eps_list.append(eps)
    
    except AttributeError:
        pass
    except IndexError:
        pass

AttributeError: 'NoneType' object has no attribute 'find_all'

In [23]:
# remove B and convert Mkt_cap to Float
new_cap_list = []
for cap in mkt_cap_list:
    try:
        new_cap = cap.replace('B', '')
        new_cap = float(new_cap)
        new_cap_list.append(new_cap)
    except ValueError:
        pass
mkt_cap_list = new_cap_list
print(mkt_cap_list)

'231,615'

In [25]:
# remove comma from Volume values
new_vol_list = []
for vol in volume_list:
    try:
        new_vol = vol.replace(',', '')
        new_vol = float(new_vol)
        new_vol_list.append(new_vol)
    except AttributeError:
        pass
volume_list = new_vol_list
# print(volume_list)

ValueError: could not convert string to float: '449,380'

In [ ]:
# create and initialize list 
company_name_list = []
company_desc_list = []
company_tel_list = []
company_addr_list = []
key_execs_n_list = []
key_execs_t_list = []
key_execs_s_list = []
url_list = []

In [ ]:
for ticker in tickers:
    try:
        # Visit Yahoo finance profile page
        url2 = base_url + ticker + '/profile?p=' + ticker
    #     print(url2)
        browser.visit(url2)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")

        profile1 = soup.find('p', class_="D(ib) W(47.727%) Pend(40px)")
        company_addr = profile1.contents[1] + profile1.contents[5]
        company_addr_list.append(company_addr)
    #     print(company_addr)
        #type(data)
        company_name = soup.find("h3", class_="Fz(m) Mb(10px)").text
        company_name_list.append(company_name)
        company_desc = soup.find('p', class_="Mt(15px) Lh(1.6)").text
        company_desc_list.append(company_desc)
    #     print(company_desc)
        key_execs_data = soup.find('table', class_='W(100%)').find('tbody').find_all('span')
#         print(key_execs_data[0])
        key_execs_name = key_execs_data[0].contents[1]
        key_execs_title = key_execs_data[1].contents[1]
        key_execs_salary = key_execs_data[2].contents[1] 
        if len(key_execs_salary):
            key_execs_salary = 'N/A'
        key_execs_n_list.append(key_execs_name)
        key_execs_t_list.append(key_execs_title)
        key_execs_s_list.append(key_execs_salary)
        profile = soup.find('p', class_="D(ib) W(47.727%) Pend(40px)").find_all('a')
        company_tel = profile[0].text
        url = profile[1].text
        company_tel_list.append(company_tel)
        url_list.append(url)
        
    except IndexError as e:
        pass
    except AttributeError as e:
        pass

In [ ]:
# convert telephone number to Float
new_tel_list = []
for tel in company_tel_list:
    try:
        new_tel = tel.replace('-', '')
        new_tel = float(new_tel)
        new_tel_list.append(new_tel)
    except ValueError:
        pass
company_tel_list = new_tel_list
#print(company_tel_list)

In [ ]:
# remove M and convert Key_Execs_Salary to Float
new_s_list = []
for s in key_execs_s_list:
    try:
        new_s = s.replace('M', '')
        new_s = float(new_s)
        new_s_list.append(new_s)
        
    except ValueError:
        pass
    
key_execs_s_list = new_s_list
# print(key_execs_s_list)

In [ ]:
# create and initialize list 
total_ESG_score_list = []
avg_perf_env_list = []
social_score_list = []
governance_score_list = []

In [ ]:
for ticker in tickers:
    try:
        # Visit Yahoo finance sustanability page
        url3 = base_url + ticker + '/sustainability?p=' + ticker
    #     print(url3)
        browser.visit(url3)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")
        sustain = soup.find_all('div', class_="D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)")
    #     print(sustain[count].contents[0])
    #     count += 1
        total_ESG_score = soup.find('div', class_="Fz(36px) Fw(600) D(ib) Mend(5px)").text
        total_ESG_score_list.append(total_ESG_score)
        avg_perf_env = sustain[0].contents[0]
        avg_perf_env_list.append(avg_perf_env)
        social_score = sustain[1].contents[0]
        social_score_list.append(social_score)
        governance_score = sustain[2].contents[0]
        governance_score_list.append(governance_score)
        
    except AttributeError as e:
        pass

In [ ]:
# Create Summary dataframe
data2_dic = {'Ticker':tickers, 'Close_Price':close_price_list, 'Volume':volume_list,'Mkt_Cap':mkt_cap_list, 'EPS':eps_list, 'Earnings_Date':earnings_Date_list}
summary_df = pd.DataFrame.from_dict(data2_dic, orient='index')
summary_df = summary_df.transpose()
summary_df

In [ ]:
# Create Profile dataframe
data_dic = {'Ticker':tickers, 'Company_Name':company_name_list, 'Description':company_desc_list, 'Telephone':company_tel_list, 'Address':company_addr_list,'Key_Execs_Name':key_execs_n_list,'Key_Execs_Title':key_execs_t_list,'Web_URL':url_list}
profile_df = pd.DataFrame.from_dict(data_dic, orient='index')
profile_df = profile_df.transpose()
profile_df

In [ ]:
# Create Combine Dataframe summary and profile
profile_df = profile_df.drop(columns=['Ticker'])
summary_profile_combine_df = pd.concat([profile_df, summary_df], axis=1, sort=True)
summary_profile_combine_df

In [ ]:
# Create sustainability dataframe
data1_dic = {'Ticker':tickers, 'Total_ESG_Score':total_ESG_score_list, 'AVG_Perf_Env':avg_perf_env_list,'Social_Score':social_score_list, 'Governance_Score':governance_score_list}
sustainability_df = pd.DataFrame.from_dict(data1_dic, orient='index')
sustainability_df = sustainability_df.transpose()

In [ ]:
# Add sustainability to combine dataframe
sustainability_df = sustainability_df.drop(columns=['Ticker'])
summary_profile_sustainability_df = pd.concat([summary_profile_combine_df, sustainability_df], axis=1, sort=True)
summary_profile_sustainability_df

## Part 3: Loading into Database Mysql

In [ ]:
# Database 
rds_connection_string = "root:root123@127.0.0.1:3306/twitter_stocks_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
# converted Combine DataFrame into database table stock_data
summary_profile_sustainability_df.to_sql(name='stock_data', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from stock_data', con=engine).head()

In [ ]:
# converted Summary DataFrame into database table summary
summary_df.to_sql(name='summary', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from summary', con=engine).head()

In [ ]:
# converted profile DataFrame into database table profile
# pd.to_sql('TRUNCATE TABLE profile', con=engine)
profile_df.to_sql(name='profile', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from profile', con=engine).head()

In [ ]:
# converted sustainability DataFrame into database table sustainability
sustainability_df.to_sql(name='sustainability', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from sustainability', con=engine).head()